<a href="https://colab.research.google.com/github/gawankarsanket/dswithchirag_assignments/blob/main/RandomForest_Assignment_Metabolic_Syndrome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset
https://www.kaggle.com/datasets/antimoni/metabolic-syndrome

In [2]:
import pandas as pd
import numpy as np

df=pd.read_csv('/content/Metabolic Syndrome.csv')

df.head()

,seqn,Age,Sex,Marital,Income,Race,WaistCirc,BMI,Albuminuria,UrAlbCr,UricAcid,BloodGlucose,HDL,Triglycerides,MetabolicSyndrome
0,62161,22,Male,Single,8200.0,White,81.0,23.3,0,3.88,4.9,92,41,84,No MetSyn
1,62164,44,Female,Married,4500.0,White,80.1,23.2,0,8.55,4.5,82,28,56,No MetSyn
2,62169,21,Male,Single,800.0,Asian,69.6,20.1,0,5.07,5.4,107,43,78,No MetSyn
3,62172,43,Female,Single,2000.0,Black,120.4,33.3,0,5.22,5.0,104,73,141,No MetSyn
4,62177,51,Male,Married,NaN,Asian,81.1,20.1,0,8.13,5.0,95,43,126,No MetSyn


In [4]:
df.shape

(2401, 15)

In [5]:
df.isnull().sum()

seqn                   0
Age                    0
Sex                    0
Marital              208
Income               117
Race                   0
WaistCirc             85
BMI                   26
Albuminuria            0
UrAlbCr                0
UricAcid               0
BloodGlucose           0
HDL                    0
Triglycerides          0
MetabolicSyndrome      0
dtype: int64

In [6]:
df.duplicated().sum()

0

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2401 entries, 0 to 2400
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   seqn               2401 non-null   int64  
 1   Age                2401 non-null   int64  
 2   Sex                2401 non-null   object 
 3   Marital            2193 non-null   object 
 4   Income             2284 non-null   float64
 5   Race               2401 non-null   object 
 6   WaistCirc          2316 non-null   float64
 7   BMI                2375 non-null   float64
 8   Albuminuria        2401 non-null   int64  
 9   UrAlbCr            2401 non-null   float64
 10  UricAcid           2401 non-null   float64
 11  BloodGlucose       2401 non-null   int64  
 12  HDL                2401 non-null   int64  
 13  Triglycerides      2401 non-null   int64  
 14  MetabolicSyndrome  2401 non-null   object 
dtypes: float64(5), int64(6), object(4)
memory usage: 281.5+ KB


Target feature is "MetabolicSyndrome".
Need to convert to numeric using label encoder

In [12]:
# SPLIT X and Y

x=df.iloc[:,1:14]
y=df.iloc[:,14:]

#Label Encoder for Target Y

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

label_encoder=preprocessing.LabelEncoder()

y['MetabolicSyndrome']=label_encoder.fit_transform(y['MetabolicSyndrome'])


#SPLIT train and Test

from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.1,random_state=10)


xtrain.shape,xtest.shape,ytrain.shape,ytest.shape

((2160, 13), (241, 13), (2160, 1), (241, 1))

## AUTOMATION
using pipeline for multiple task like OHE, scaling etc

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder # OHE for input feature
from sklearn.preprocessing import StandardScaler # Normalise data
from sklearn.impute import SimpleImputer    # Handle missing value
from sklearn.compose import ColumnTransformer  #To apply  separate transformers for numerical and categorical data


In [17]:
# Define separate list of columns as numeric and object

lst_num=[]
lst_cat=[]

for i in range(len(x.columns)):
  if x.dtypes[i]=='object':
    lst_cat.append(x.columns[i])

  else:
    lst_num.append(x.columns[i])



In [21]:
## Feature Engg Automation

#Create pipeline to process Cat data and Num data.

num_pipeline=Pipeline(
    steps=[
        ('Imputer',SimpleImputer(strategy='mean')), #Handling missing values
        ('Scaling',StandardScaler()) ##Feature Scaling/Normalisation
    ]
)


cat_pipeline=Pipeline(
    steps=[
        ('Imputer',SimpleImputer(strategy='most_frequent')), #Handling missing value
        ('OHE',OneHotEncoder())  # Categorical to numeric
    ]
)



In [23]:
preprocessor=ColumnTransformer([
    ('numpipeline',num_pipeline,lst_num),
    ('catpipeline',cat_pipeline,lst_cat)
])

preprocessor

ColumnTransformer(transformers=[('numpipeline',
                                 Pipeline(steps=[('Imputer', SimpleImputer()),
                                                 ('Scaling',
                                                  StandardScaler())]),
                                 ['Age', 'Income', 'WaistCirc', 'BMI',
                                  'Albuminuria', 'UrAlbCr', 'UricAcid',
                                  'BloodGlucose', 'HDL', 'Triglycerides']),
                                ('catpipeline',
                                 Pipeline(steps=[('Imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('OHE', OneHotEncoder())]),
                                 ['Sex', 'Marital', 'Race'])])

In [24]:
#Apply scaling , Imputation, encoding to x data as defined in pipeline >> Preprocessor
#as defined in


xtrain=preprocessor.fit_transform(xtrain)
xtest=preprocessor.transform(xtest)

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

## Model Training Automation
models={
    'Random Forest':RandomForestClassifier(),
    'Decision Tree':DecisionTreeClassifier()

}

In [26]:
from sklearn.metrics import accuracy_score


def evaluate_model (xtrain,ytrain,xtest,ytest,models):
  report={}
  for i in range(len(models)):
    model=list(models.values())[i]
    model.fit(xtrain, ytrain)


    preds=model.predict(xtest)

    score=accuracy_score(ytest,preds)

    report[list(models.keys())[i]] = score



  return report

In [27]:
evaluate_model(xtrain,ytrain,xtest,ytest,models)

<ipython-input-26-c64bf8e2e8f1>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(xtrain, ytrain)


{'Random Forest': 0.9045643153526971, 'Decision Tree': 0.8547717842323651}

### HYPER PARAMETER TUNING

In [28]:
#Hyperparameter tuning, try diff values of 'n_estimators', 'max_depth'
# and see how model performs at each stage

#separate object for each method
classifier_RF=RandomForestClassifier()
classifier_DT=DecisionTreeClassifier()


#separate set of parameters for each method
## Hypeparameter Tuning for RandomForest
params_RF={'max_depth':[3,5,10,None],
        'n_estimators':[10,50,100],
        'criterion':['gini','entropy'],
        'oob_score':[True]
}

## Hypeparameter Tuning for DT
params_DT={'max_depth':[3,5,10,None],
           'criterion':['gini','entropy']
           }


In [30]:
from sklearn.model_selection import RandomizedSearchCV

#testing for RF methods, as input given is RF classifier and RF Parameters

cv=RandomizedSearchCV(classifier_RF,param_distributions=params_RF,scoring='accuracy',cv=5,verbose=3)
cv.fit(xtrain,ytrain)

cv.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=5, n_estimators=10, oob_score=True;, score=0.866 total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


[CV 2/5] END criterion=gini, max_depth=5, n_estimators=10, oob_score=True;, score=0.882 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=5, n_estimators=10, oob_score=True;, score=0.852 total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


[CV 4/5] END criterion=gini, max_depth=5, n_estimators=10, oob_score=True;, score=0.870 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=5, n_estimators=10, oob_score=True;, score=0.854 total time=   0.2s


...

[CV 4/5] END criterion=gini, max_depth=3, n_estimators=100, oob_score=True;, score=0.861 total time=   0.3s


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV 5/5] END criterion=gini, max_depth=3, n_estimators=100, oob_score=True;, score=0.854 total time=   0.3s


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'oob_score': True, 'n_estimators': 100, 'max_depth': 10, 'criterion': 'gini'}

In [32]:
#Validation for DT
cv_dt=RandomizedSearchCV(classifier_DT,param_distributions=params_DT,scoring='accuracy',cv=5,verbose=3)
cv_dt.fit(xtrain,ytrain)

cv_dt.best_params_

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END .......criterion=gini, max_depth=3;, score=0.877 total time=   0.0s
[CV 2/5] END .......criterion=gini, max_depth=3;, score=0.854 total time=   0.0s
[CV 3/5] END .......criterion=gini, max_depth=3;, score=0.850 total time=   0.0s
[CV 4/5] END .......criterion=gini, max_depth=3;, score=0.866 total time=   0.0s
[CV 5/5] END .......criterion=gini, max_depth=3;, score=0.852 total time=   0.0s
[CV 1/5] END .......criterion=gini, max_depth=5;, score=0.856 total time=   0.0s
[CV 2/5] END .......criterion=gini, max_depth=5;, score=0.859 total time=   0.0s
[CV 3/5] END .......criterion=gini, max_depth=5;, score=0.863 total time=   0.0s
[CV 4/5] END .......criterion=gini, max_depth=5;, score=0.861 total time=   0.0s
[CV 5/5] END .......criterion=gini, max_depth=5;, score=0.840 total time=   0.0s
[CV 1/5] END ......criterion=gini, max_depth=10;, score=0.859 total time=   0.0s
[CV 2/5] END ......criterion=gini, max_depth=10;,

{'max_depth': 5, 'criterion': 'entropy'}